# 🧪 Análisis de Outliers - Versión Avanzada
Este notebook muestra cómo detectar y tratar outliers utilizando distintas estrategias adaptadas a cada variable del dataset. Incluye detección con IQR y Z-score, y tratamientos como eliminación, winsorización y sustitución por mediana.

In [2]:
# Crear un DataFrame con outliers incluidos
import pandas as pd

# Generamos datos base para 100 registros
import numpy as np
np.random.seed(42)

data_outliers = {
    'ID': list(range(1, 101)),
    'Edad': np.random.normal(loc=35, scale=5, size=100).astype(int).tolist(),  # Edad promedio ~35
    'Ingresos': np.random.normal(loc=2000, scale=300, size=100).astype(int).tolist(),  # Ingresos promedio ~2000€
    'Tiempo_uso': np.random.normal(loc=2.5, scale=0.5, size=100).round(2).tolist(),  # Horas promedio ~2.5
    'Genero': ['M', 'F'] * 50
}

# Insertamos outliers manualmente en Edad
data_outliers['Edad'][5] = 99   # Muy alto
data_outliers['Edad'][15] = 3   # Muy bajo

# Insertamos outliers en Ingresos
data_outliers['Ingresos'][10] = 10000  # Muy alto
data_outliers['Ingresos'][20] = 100    # Muy bajo

# Insertamos outliers en Tiempo_uso
data_outliers['Tiempo_uso'][25] = 15.0
data_outliers['Tiempo_uso'][30] = 20.0

df = pd.DataFrame(data_outliers)


## Detección de outliers con IQR
El rango intercuartílico (IQR) es una técnica robusta frente a valores extremos que identifica outliers en una distribución sin asumir normalidad.

Q1: percentil 25 → marca el límite inferior de los valores centrales.

Q3: percentil 75 → marca el límite superior de los valores centrales.

IQR: diferencia entre Q3 y Q1 → mide la dispersión del 50% central de los datos.

Se considera outlier cualquier valor fuera del rango [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR].

In [3]:
# 📊 Detección de outliers con IQR
for col in ['Edad', 'Ingresos', 'Tiempo_uso']:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    print(f"{col} - IQR Outliers:", df[(df[col] < lower) | (df[col] > upper)].shape[0])


Edad - IQR Outliers: 3
Ingresos - IQR Outliers: 3
Tiempo_uso - IQR Outliers: 4


## Detección de outliers con Z-score
El Z-score mide cuántas desviaciones estándar se aleja un valor respecto a la media. Se basa en la fórmula:
Si el valor de Z está por encima de 3 o por debajo de -3, se considera un outlier extremo (siguiendo la regla empírica de la normal).
Esta técnica asume distribución normal, por lo que es menos robusta ante datos asimétricos.

In [ ]:
# 📏 Detección de outliers con Z-score
from scipy.stats import zscore
import numpy as np

for col in ['Edad', 'Ingresos', 'Tiempo_uso']:
    z_scores = zscore(df[col])
    print(f"{col} - Z-score Outliers:", (np.abs(z_scores) > 3).sum())


## Copia del DataFrame original para tratamiento
Antes de realizar cualquier transformación, es buena práctica trabajar sobre una copia del DataFrame original para no modificar los datos fuente. Esto permite comparar antes y después de aplicar los tratamientos de outliers.

In [4]:
df_tratado = df.copy()

## Eliminación de outliers en Edad (IQR)
Para la variable Edad, se usa el método del IQR para detectar outliers y se eliminan directamente las filas que contienen esos valores atípicos.

Se calcula el rango intercuartílico.

Se definen los límites inferior y superior.

Se filtran los registros que están dentro del rango considerado "normal".

In [5]:
# 1. Edad → eliminación con IQR
q1_e, q3_e = df['Edad'].quantile([0.25, 0.75])
iqr_e = q3_e - q1_e
lim_inf_e = q1_e - 1.5 * iqr_e
lim_sup_e = q3_e + 1.5 * iqr_e
df_tratado = df_tratado[(df_tratado['Edad'] >= lim_inf_e) & (df_tratado['Edad'] <= lim_sup_e)]


## Winsorización en Ingresos (P5-P95)
La winsorización consiste en limitar los valores extremos a unos percentiles determinados. Aquí se ajustan los valores de Ingresos para que estén entre el percentil 5 y el percentil 95:

Valores por debajo del P5 se sustituyen por el valor del P5.

Valores por encima del P95 se sustituyen por el valor del P95.

Este método suaviza los extremos sin eliminar filas.

In [ ]:
# 2. Ingresos → winsorización entre P5 y P95
p5, p95 = df['Ingresos'].quantile([0.05, 0.95])
df_tratado['Ingresos'] = df_tratado['Ingresos'].clip(lower=p5, upper=p95)


## Sustitución por mediana en Tiempo_uso
Para la variable Tiempo_uso, los valores fuera del rango IQR se reemplazan por la mediana. Este enfoque conserva todas las filas pero reduce el impacto de los extremos, especialmente útil en datos sesgados:

Se identifican outliers usando IQR.

En lugar de eliminar o truncar, se sustituye por la mediana de la columna.

python
Copiar
Editar


In [6]:
# 3. Tiempo_uso → reemplazo por mediana si fuera del IQR
q1_t, q3_t = df['Tiempo_uso'].quantile([0.25, 0.75])
iqr_t = q3_t - q1_t
lim_inf_t = q1_t - 1.5 * iqr_t
lim_sup_t = q3_t + 1.5 * iqr_t
mediana_t = df['Tiempo_uso'].median()
df_tratado.loc[
    (df_tratado['Tiempo_uso'] < lim_inf_t) | (df_tratado['Tiempo_uso'] > lim_sup_t),
    'Tiempo_uso'
] = mediana_t


In [ ]:
import matplotlib.pyplot as plt

variables = ['Edad', 'Ingresos', 'Tiempo_uso']
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(variables):
    axs[i].boxplot([df[col], df_tratado[col]], labels=['Original', 'Tratado'])
    axs[i].set_title(f'Boxplot - {col}')

plt.tight_layout()
plt.show()


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(variables):
    axs[i].hist(df[col], bins=20, alpha=0.5, label='Original')
    axs[i].hist(df_tratado[col], bins=20, alpha=0.5, label='Tratado')
    axs[i].set_title(f'Histograma - {col}')
    axs[i].legend()

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 3, figsize=(18, 5))
variables = ['Edad', 'Ingresos', 'Tiempo_uso']

for i, col in enumerate(variables):
    sns.histplot(df[col], kde=True, stat="density", bins=20, ax=axs[i], label="Original", color="skyblue", alpha=0.5)
    sns.histplot(df_tratado[col], kde=True, stat="density", bins=20, ax=axs[i], label="Tratado", color="orange", alpha=0.5)
    axs[i].set_title(f'Histograma - {col}')
    axs[i].legend()

plt.tight_layout()
plt.show()


In [ ]:
# ✅ Validación final del dataset tratado
print(df_tratado.shape)
df_tratado.describe()